In [1]:
import gym
import os

from datetime import datetime
from tensorforce import Environment, Agent, Runner
from tensorforce_pacman_env import tf_pacman_env

import ale_py.roms as roms
import tensorflow as tf
import tf_agents as tf_a
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Counts # of trace files in folder
_, _, files = next(os.walk("./pacman_traces"))
trace_count = len(files)

environment = Environment.create(environment=tf_pacman_env)
print(environment.reset())
agent = Agent.create(agent='ddqn', environment=environment, batch_size = 1, memory = 10000) 
print("Agent Created")
agent.pretrain(directory='pacman_traces', num_iterations=30, num_traces=trace_count, num_updates=1)
print("Agent pretrained")
environment.reset()

Agent Created
Agent pretrained


In [3]:
# env_name = 'ALE/MsPacman-v5'

# env = gym.make(env_name)
# env = gym.wrappers.AtariPreprocessing(
#     env,
#     noop_max = 30, 
#     frame_skip = 1, 
#     screen_size = 84, 
#     terminal_on_life_loss = False,
#     grayscale_obs = True, 
#     grayscale_newaxis = False, 
#     scale_obs = False)
# env = gym.wrappers.FrameStack(env, num_stack = 4)

environment._expect_receive = None
environment._actions = None

runner = Runner(agent=agent, environment=environment)
runner.run(num_episodes=100, evaluation=True)

Episodes: 100%|██████████| 100/100 [01:26, return=300.00, ts/ep=492, sec/ep=1.33, ms/ts=2.7, agent=69.3%]

In [6]:
environment._expect_receive = None
environment._actions = None

new_agent = Agent.create(agent='ddqn', environment=environment, batch_size = 1, memory = 10000)
new_runner = Runner(agent=new_agent, environment=environment)
new_runner.run(num_episodes=100, evaluation=True)

Episodes: 100%|██████████| 100/100 [03:37, return=70.00, ts/ep=504, sec/ep=1.49, ms/ts=3.0, agent=69.7%]





































































































